This notebook demonstrates different `shapes` attributions in Edward's `RandomVariable` variables. 

See the API doc for more detail: http://edwardlib.org/api/ed/RandomVariable


In [1]:
import numpy as np
import edward as ed
import tensorflow as tf

import scipy.stats

A random variable of type `RandomVariable` has shape defined as : `sample_shape + batch_shape + event_shape`

- `sample_shape`: represents the dimensions of samples drawn from the distribution

- `batch_shape`: number of independent variables, determined by the _broadcasted_ result of the shapes of the parameters

- `event_shape`: the shape of one draw from the distribution



In [2]:
list_of_alphas = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
list_of_betas = [2.0, 4.0, 6.0]

ed_beta_dist = ed.models.Beta(list_of_alphas, list_of_betas)

In [3]:
ed_beta_dist.sample_shape, ed_beta_dist.batch_shape, ed_beta_dist.event_shape

(TensorShape([]), TensorShape([Dimension(2), Dimension(3)]), TensorShape([]))

In [4]:
print("sample shape = {0:}".format(ed_beta_dist.sample_shape))
print("batch shape = {0:}".format(ed_beta_dist.batch_shape))
print("event_shape shape = {0:}".format(ed_beta_dist.event_shape))

sample shape = ()
batch shape = (2, 3)
event_shape shape = ()


In the above demo, the `sample_shape` is a scalar; `event_shape` is also a scalar because the output of a Beta distribution is a scalar. `batch_shape` is determined by the broadcasted result of `list_of_alphas`, which is 2x3, and `list_of_betas`, which is 1x3. The broadcasted shape is 2x3.

Therefore, the parameter will be broadcasted to:

\begin{equation}
\text{params} = \bigg[
\underbrace{\begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6
\end{pmatrix}}_{\alpha},
\underbrace{\begin{pmatrix}
2 & 4 & 6 \\
2 & 4 & 6
\end{pmatrix}}_{\beta}
\bigg]
\end{equation}

For compariosn and check, we will also create an equivalent scipy version of the Beta distribution

In [5]:
scipy_beta_dist = scipy.stats.beta(list_of_alphas, list_of_betas)

### Below we use different dimensions of test points to demonstrate different scenarios

In [6]:
sess = ed.get_session()

#### 1. When the test point `x` has the same dimension as the parameter:

\begin{pmatrix}
0.431... & 0.705... & 0.922...\\
0.683... & 0.766... & 0.403...
\end{pmatrix}

There is a straightfoward element-wise correspondence between the test point and the parameter.

In [62]:
xs = np.random.random((2, 3)).tolist()
print("xs = {0:}\n".format(xs))
print("scipy output:\n {0:}\n".format(scipy_beta_dist.pdf(xs)))
print("Edward output:\n {0:}\n".format(sess.run(ed_beta_dist.prob(xs))))


xs = [[0.43180560496131903, 0.7052561828103878, 0.9222299365765809], [0.6836853507203573, 0.7668789899782873, 0.4031126203151161]]

scipy output:
 [[  1.13638879e+00   3.61169451e-01   4.06490106e-04]
 [  2.02170658e+00   1.22690133e+00   2.23556439e+00]]

Edward output:
 [[  1.13638878e+00   3.61169517e-01   4.06489911e-04]
 [  2.02170682e+00   1.22690177e+00   2.23556376e+00]]



In [65]:
print("the (1,1) element should be {0:}".format(scipy.stats.beta(5, 4).pdf(0.7668789899782873,)))

the (1,1) element should be 1.2269013325347171


#### 2. When the test point `x` has "smaller" dimension than the parameter:

\begin{pmatrix}
0.855... & 0.530... & 0.566...
\end{pmatrix}

The test point will be broadcasted to match the dimension of the parameters.

For example, given the above test point and parameter, the test point will first be enlarged to

\begin{pmatrix}
0.855... & 0.530... & 0.566... \\
0.855... & 0.530... & 0.566... 
\end{pmatrix}


In [59]:
xs = np.random.random((1, 3)).tolist()
print("xs = {0:}\n".format(xs))
print("scipy output:\n {0:}\n".format(scipy_beta_dist.pdf(xs)))
print("Edward output:\n {0:}\n".format(sess.run(ed_beta_dist.prob(xs))))


xs = [[0.855730095527968, 0.530398195874685, 0.5668700545195146]]

scipy output:
 [[ 0.28853981  1.09855373  0.82293847]
 [ 1.80807382  2.29485608  2.47344272]]

Edward output:
 [[ 0.2885398   1.0985539   0.8229385 ]
 [ 1.80807376  2.29485559  2.47344279]]



In [66]:
print("The (1,2) element should be {0:}".format(scipy.stats.beta(6, 6).pdf(0.5668700545195146)))

The (1,2) element should be 2.4734427205441802


#### 3. When the test point `x` has "larger" dimension than the parameter:

Both scipy and edward will return error


In [72]:
xs = np.random.random((4, 3)).tolist()
scipy_beta_dist.pdf(xs)

ValueError: operands could not be broadcast together with shapes (2,3) (4,3) 

In [73]:
sess.run(ed_beta_dist.prob(np.random.random((4, 3)).tolist()))

ValueError: Dimensions must be equal, but are 2 and 4 for 'Beta_1/prob_27/mul' (op: 'Mul') with input shapes: [2,3], [4,3].